<img src="../../../../../images/qiskit_header.png" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" align="middle">

# _*Qiskit Finance: Portfolio Optimization*_ 

The latest version of this notebook is available on https://github.com/Qiskit/qiskit-tutorials.

***
### Contributors
Stefan Woerner<sup>[1]</sup>, Daniel Egger<sup>[1]</sup>, Shaohan Hu<sup>[1]</sup>, Stephen Wood<sup>[1]</sup>, Marco Pistoia<sup>[1]</sup>
### Affliation
- <sup>[1]</sup>IBMQ

### Introduction

This tutorial shows how to solve the following mean-variance portfolio optimization problem for $n$ assets:

$\begin{aligned}
\min_{x \in \{0, 1\}^n}  q x^T \Sigma x - \mu^T x\\
\text{subject to: } 1^T x = B
\end{aligned}$

where we use the following notation:

- $x \in \{0, 1\}^n$ denotes the vector of binary decision variables, which indicate which assets to pick ($x[i] = 1$) and which not to pick ($x[i] = 0$),
- $\mu \in \mathbb{R}^n$ defines the expected returns for the assets,
- $\Sigma \in \mathbb{R}^{n \times n}$ specifies the covariances between the assets,
- $q > 0$ controls the risk appetite of the decision maker,
- and $B$ denotes the budget, i.e. the number of assets to be selected out of $n$.

We assume the following simplifications:
- all assets have the same price (normalized to 1),
- the full budget $B$ has to be spent, i.e. one has to select exactly $B$ assets.

The equality constraint $1^T x = B$ is mapped to a penalty term $(1^T x - B)^2$ which is scaled by a parameter and subtracted from the objective function. 
The resulting problem can be mapped to a Hamiltonian whose groundstate corresponds to  the optimal solution.
This notebook shows how to use the Variational Quantum Eigensolver (VQE) or the Quantum Approximate Optimization Algorithm (QAOA) to find the optimal solution for a given set of parameters.

In [1]:
from qiskit import BasicAer
from qiskit.aqua import QuantumInstance
from qiskit.aqua import Operator, run_algorithm
from qiskit.aqua.input import EnergyInput
from qiskit.aqua.translators.ising import portfolio
from qiskit.aqua.translators.data_providers import RandomDataProvider
from qiskit.aqua.algorithms import VQE, QAOA, ExactEigensolver
from qiskit.aqua.components.optimizers import COBYLA
from qiskit.aqua.components.variational_forms import RY
import numpy as np
import datetime

### [Optional] Setup token to run the experiment on a real device
If you would like to run the experiement on a real device, you need to setup your account first.

Note: If you do not store your token yet, use `IBMQ.save_accounts()` to store it first.

In [ ]:
from qiskit import IBMQ
IBMQ.load_accounts()

### Define problem instance

Here an Operator instance is created for our Hamiltonian. In this case the paulis are from an Ising Hamiltonian translated from the portfolio problem. We use a random portfolio problem for this notebook. It is straight-forward to extend this to using real financial data as illustrated here:<br>
[Loading and Processing Stock-Market Time-Series Data](../data_providers/time_series.ipynb)

In [2]:
# set number of assets (= number of qubits)
num_assets = 4

# Generate expected return and covariance matrix from (random) time-series
stocks = [("TICKER%s" % i) for i in range(num_assets)]
data = RandomDataProvider(tickers=stocks,
                 start=datetime.datetime(2016,1,1),
                 end=datetime.datetime(2016,1,30))
data.run()
mu = data.get_period_return_mean_vector()
sigma = data.get_period_return_covariance_matrix()

In [3]:
q = 0.5 # set risk factor
budget = int(num_assets / 2) # set budget
penalty = num_assets # set parameter to scale the budget penalty term

qubitOp, offset = portfolio.get_portfolio_qubitops(mu, sigma, q, budget, penalty)
algo_input = EnergyInput(qubitOp)

We define some utility methods to print the results in a nice format.

In [4]:
def index_to_selection(i, num_assets):
    s = "{0:b}".format(i).rjust(num_assets)
    x = np.array([1 if s[i]=='1' else 0 for i in reversed(range(num_assets))])
    return x

def print_result(result):
    selection = portfolio.sample_most_likely(result['eigvecs'][0])
    value = portfolio.portfolio_value(selection, mu, sigma, q, budget, penalty)
    print('Optimal: selection {}, value {:.4f}'.format(selection, value))

    probabilities = np.abs(result['eigvecs'][0])**2
    i_sorted = reversed(np.argsort(probabilities))
    print('\n----------------- Full result ---------------------')
    print('selection\tvalue\t\tprobability')
    print('---------------------------------------------------')
    for i in i_sorted:
        x = index_to_selection(i, num_assets)
        value = portfolio.portfolio_value(x, mu, sigma, q, budget, penalty)    
        probability = probabilities[i]
        print('%10s\t%.4f\t\t%.4f' %(x, value, probability))

### ExactEigensolver (as a classical reference)
Lets solve the problem. First classically...

We can now use the Operator we built above without regard to the specifics of how it was created. To run an algorithm we need to prepare a configuration params dictionary. We set the algorithm for the ExactEigensolver so we can have a classical reference. The problem is set for 'ising'. Backend is not required since this is computed classically not using quantum computation. The params, along with the algo input containing the operator, are now passed to the algorithm to be run. The result is returned as a dictionary.

In [5]:
exact_eigensolver = ExactEigensolver(qubitOp, k=1)
result = exact_eigensolver.run()

""" the equivalent if using declarative approach
algorithm_cfg = {
    'name': 'ExactEigensolver'
}

params = {
    'problem': {'name': 'ising'},
    'algorithm': algorithm_cfg
}
result = run_algorithm(params, algo_input)
"""

print_result(result)

Optimal: selection [0 0 1 1], value -0.0026

----------------- Full result ---------------------
selection	value		probability
---------------------------------------------------
 [0 0 1 1]	-0.0026		1.0000
 [1 1 1 1]	15.9996		0.0000
 [0 1 1 1]	3.9984		0.0000
 [1 0 1 1]	3.9985		0.0000
 [1 1 0 1]	4.0000		0.0000
 [0 1 0 1]	-0.0011		0.0000
 [1 0 0 1]	-0.0011		0.0000
 [0 0 0 1]	3.9978		0.0000
 [1 1 1 0]	4.0017		0.0000
 [0 1 1 0]	0.0006		0.0000
 [1 0 1 0]	0.0006		0.0000
 [0 0 1 0]	3.9995		0.0000
 [1 1 0 0]	0.0021		0.0000
 [0 1 0 0]	4.0010		0.0000
 [1 0 0 0]	4.0011		0.0000
 [0 0 0 0]	16.0000		0.0000


### Solution using VQE
We can now use the Variational Quantum Eigensolver (VQE) to solve the problem. We will specify the optimizer and variational form to be used.

Note: You can switch to different backends by providing the name of backend.

In [6]:
backend = BasicAer.get_backend('statevector_simulator')
seed = 50

cobyla = COBYLA()
cobyla.set_options(maxiter=500)
ry = RY(qubitOp.num_qubits, depth=3, entanglement='full')
vqe = VQE(qubitOp, ry, cobyla)
vqe.random_seed = seed

quantum_instance = QuantumInstance(backend=backend, seed=seed, seed_transpiler=seed)

result = vqe.run(quantum_instance)

"""declarative approach
algorithm_cfg = {
    'name': 'VQE',
    'operator_mode': 'matrix'
}

optimizer_cfg = {
    'name': 'COBYLA',
    'maxiter': 500
}

var_form_cfg = {
    'name': 'RY',
    'depth': 3,
    'entanglement': 'full'
}

params = {
    'problem': {'name': 'ising', 'random_seed': seed},
    'algorithm': algorithm_cfg,
    'optimizer': optimizer_cfg,
    'variational_form': var_form_cfg
}
result = run_algorithm(params, algo_input, backend=backend)
"""
print_result(result)

Optimal: selection [0 1 1 0], value 0.0006

----------------- Full result ---------------------
selection	value		probability
---------------------------------------------------
 [0 1 1 0]	0.0006		0.7038
 [1 0 0 1]	-0.0011		0.2120
 [1 0 1 0]	0.0006		0.0272
 [0 1 0 1]	-0.0011		0.0251
 [1 1 0 0]	0.0021		0.0167
 [0 0 1 1]	-0.0026		0.0151
 [1 0 0 0]	4.0011		0.0000
 [0 1 1 1]	3.9984		0.0000
 [0 0 0 1]	3.9978		0.0000
 [1 0 1 1]	3.9985		0.0000
 [0 1 0 0]	4.0010		0.0000
 [1 1 1 0]	4.0017		0.0000
 [1 1 0 1]	4.0000		0.0000
 [0 0 0 0]	16.0000		0.0000
 [0 0 1 0]	3.9995		0.0000
 [1 1 1 1]	15.9996		0.0000


### Solution using QAOA

We also show here a result using the Quantum Approximate Optimization Algorithm (QAOA). This is another variational algorithm and it uses an internal variational form that is created based on the problem.

In [7]:
backend = BasicAer.get_backend('statevector_simulator')
seed = 50

cobyla = COBYLA()
cobyla.set_options(maxiter=250)
qaoa = QAOA(qubitOp, cobyla, 3)

qaoa.random_seed = seed

quantum_instance = QuantumInstance(backend=backend, seed=seed, seed_transpiler=seed)

result = qaoa.run(quantum_instance)

"""declarative approach
algorithm_cfg = {
    'name': 'QAOA.Variational',
    'p': 3,
    'operator_mode': 'matrix'
}

optimizer_cfg = {
    'name': 'COBYLA',
    'maxiter': 250
}

params = {
    'problem': {'name': 'ising', 'random_seed': seed},
    'algorithm': algorithm_cfg,
    'optimizer': optimizer_cfg
}
result = run_algorithm(params, algo_input, backend=backend)
"""
print_result(result)

Optimal: selection [1 1 0 0], value 0.0021

----------------- Full result ---------------------
selection	value		probability
---------------------------------------------------
 [1 1 0 0]	0.0021		0.1667
 [1 0 1 0]	0.0006		0.1667
 [0 1 1 0]	0.0006		0.1667
 [1 0 0 1]	-0.0011		0.1666
 [0 1 0 1]	-0.0011		0.1666
 [0 0 1 1]	-0.0026		0.1666
 [0 0 0 0]	16.0000		0.0000
 [1 1 1 1]	15.9996		0.0000
 [1 1 1 0]	4.0017		0.0000
 [1 0 0 0]	4.0011		0.0000
 [0 1 0 0]	4.0010		0.0000
 [1 1 0 1]	4.0000		0.0000
 [0 0 1 0]	3.9995		0.0000
 [1 0 1 1]	3.9985		0.0000
 [0 1 1 1]	3.9984		0.0000
 [0 0 0 1]	3.9978		0.0000
